In [1]:
%matplotlib inline

import os
import pickle
import numpy as np

import gym
import neat
import cv2

import utils.PyPlotReporter
import utils.CSVReporter
import utils.SubstrateNet as Substrate
from utils.EnvEvaluator import * 
from utils.SubstrateNet import GridMappingBounds, DirectMapping
from utils.Profiling import get_exec_times_pd
from utils.Atari import CONTROLLER_TO_ACTION,CONTROLLER_TO_ACTION_SHORT, CONTROLLER_TO_ACTION_FORCE

Using TensorFlow backend.


In [2]:
class HyperNeatEnvEvaluator(HyperNeatEnvEvaluator):
    
    def make_substrate(self):
        sub = Substrate.Sequential(min_abs_weight=0.1,bias=True)
        sub.input(Substrate.Input(GridMappingBounds((42,32), 0, range_x=(-1,1), range_y=(-1, 1))))
       # sub.add(Substrate.Dense(GridMappingBounds((21,16), 1, range_x=(-0.5,0.5), range_y=(-0.5, 0.5))))
        #                       output coords: FIRE , UP      , RIGHT, LEFT     , DOWN    , z-coord       
        sub.add(Substrate.Dense(DirectMapping([(1,0), (-0.5,1), (0,0), (-1, 0), (-0.5,-1)], 1)))
        sub.compile()
        return sub

    def preproc_img(self, observation):
        grey = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        grey = cv2.resize(grey,dsize=(32,42), interpolation=cv2.INTER_AREA) 
       # grey = (grey - np.mean(grey)) / np.std(grey) 
        grey = grey / 255
        return grey
    
    def activate_net(self, sub, observation): 
        grey = self.preproc_img(observation)
        activation = sub.activate(grey)[0]
        out = tuple(activation > .5)
        action = CONTROLLER_TO_ACTION[out]
        return action

evaluator = HyperNeatEnvEvaluator("Riverraid-v0", 2000, n_workers=8, seed=[1111])

In [3]:

config_path = "./configs/hyperatari-Copy1.cfg" 

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)


pop = neat.Population(config)
pop.add_reporter(utils.CSVReporter.CSVReporter("saves/progress.csv"))
pop.add_reporter(utils.PyPlotReporter.PyPlotReporter())
pop.add_reporter(neat.Checkpointer(1, None, "saves/cp_atari_6_"))

In [ ]:
gnome = pop.run(evaluator.eval_all_genomes, 10)

In [9]:
# 6720 parameters , max 1000 steps, cppn: 6 paramters

,name,avg_time,total_time,N
0,cppn_get_weights,0.034380,3.437980,100
1,make_cppn,0.035043,3.504271,100
2,env_make,0.119259,11.925913,100
3,substrat_forward,0.000234,110.026834,469682
4,env_step,0.000384,180.440072,469682


In [6]:
# 453264 parameters, max 10 setps

,name,avg_time,total_time,N
0,cppn_get_weights,3.084405,616.880991,200
1,make_cppn,6.177024,617.702438,100
2,env_make,0.118273,11.827256,100
3,substrat_forward,0.000300,55.337264,184542
4,env_step,0.000397,73.203334,184542


## Visualization

In [5]:
evaluator.show(gnome, config, delay=0.01, random=True)

In [ ]:
evaluator.create_video(evaluator,gnome, config, fps=60, fout="f.mp4")